In [1]:
data_dir = "UCF50/"
img_height , img_width = 64, 64
seq_len = 70
 

In [2]:
import os,cv2,pickle
import numpy as np
import IPython.display as Disp

classes = os.listdir("UCF50/")

In [3]:
import tensorflow as tf

devices = tf.config.experimental.list_physical_devices("GPU")

tf.config.experimental.set_memory_growth(devices[0],True)

In [4]:
#  Creating frames from videos
 
def frames_extraction(video_path):
    frames_list = []
     
    vidObj = cv2.VideoCapture(video_path)
    # Used as counter variable 
    count = 1
 
    while count <= seq_len: 
         
        success, image = vidObj.read() 
        if success:
            image = cv2.resize(image, (img_height, img_width))
            frames_list.append(image)
            count += 1
        else:
            print("Defected frame")
            break
 
            
    return frames_list


In [5]:
def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
            Disp.clear_output(wait = True)
            frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
            if len(frames) == seq_len:
                X.append(frames)
             
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
     
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [30]:
from keras.layers import Dense,ConvLSTM2D,Flatten,Dropout,GlobalAveragePooling2D
from keras import Sequential
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score


In [9]:
model = Sequential()
model.add(ConvLSTM2D(filters = 64, kernel_size = (3, 3), return_sequences = False, data_format = "channels_last", input_shape = (seq_len, img_height, img_width, 3)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(100, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(len(classes), activation = "softmax"))

In [10]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_lst_m2d_3 (ConvLSTM2D)  (None, 62, 62, 64)        154624    
_________________________________________________________________
dropout_4 (Dropout)          (None, 62, 62, 64)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 246016)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 100)               24601700  
_________________________________________________________________
dropout_5 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050      
Total params: 24,761,374
Trainable params: 24,761,374
Non-trainable params: 0
__________________________________________

In [8]:
X,Y = create_data("UCF50/")

YoYo


In [9]:
pickle.dump(X,open("data_X.pickle","wb"),protocol=4)
pickle.dump(Y,open("data_Y.pickle","wb"))


In [11]:
X = pickle.load(open("data_X.pickle","rb"))
Y = pickle.load(open("data_Y.pickle","rb"))

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, shuffle=True, random_state=0)

In [13]:
del X,Y

In [14]:
opt = SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])

In [15]:
earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]

In [18]:
history = model.fit(x = X_train, y = y_train, epochs=40, batch_size = 2, shuffle=True, validation_split=0.2, callbacks=callbacks)

Train on 4161 samples, validate on 1041 samples
Epoch 1/40
4161/4161 [==============================] - 611s 147ms/step - loss: 3.9260 - accuracy: 0.0252 - val_loss: 3.9095 - val_accuracy: 0.0307
Epoch 2/40
4161/4161 [==============================] - 632s 152ms/step - loss: 3.9104 - accuracy: 0.0293 - val_loss: 3.9080 - val_accuracy: 0.0307
Epoch 3/40
4161/4161 [==============================] - 635s 153ms/step - loss: 3.9067 - accuracy: 0.0293 - val_loss: 3.9065 - val_accuracy: 0.0307
Epoch 4/40
4161/4161 [==============================] - 637s 153ms/step - loss: 3.9069 - accuracy: 0.0308 - val_loss: 3.9045 - val_accuracy: 0.0307
Epoch 5/40
4161/4161 [==============================] - 640s 154ms/step - loss: 3.8970 - accuracy: 0.0341 - val_loss: 3.8177 - val_accuracy: 0.0557
Epoch 6/40
4161/4161 [==============================] - 643s 155ms/step - loss: 3.7186 - accuracy: 0.0944 - val_loss: 3.4035 - val_accuracy: 0.2037
Epoch 7/40
4161/4161 [==============================] - 646s 155

In [23]:
y_pred = model.predict(X_test)

In [40]:
print(classification_report(y_test.tolist(), y_pred.tolist(),target_names=classes))

                    precision    recall  f1-score   support

          Kayaking       0.72      0.47      0.57        38
     BaseballPitch       0.96      0.93      0.95        28
        Basketball       0.58      0.46      0.51        24
        BenchPress       1.00      0.90      0.95        40
            Biking       0.33      0.40      0.36        20
         Billiards       1.00      1.00      1.00        21
      BreastStroke       0.91      1.00      0.95        21
      CleanAndJerk       0.80      0.76      0.78        21
            Diving       0.77      0.85      0.81        27
          Drumming       0.79      0.81      0.80        27
           Fencing       0.85      0.58      0.69        19
         GolfSwing       0.79      0.87      0.83        30
          HighJump       0.50      0.56      0.53        25
         HorseRace       0.76      0.76      0.76        25
       HorseRiding       0.56      0.86      0.68        42
          HulaHoop       0.86      0.73

In [66]:
channel = "/media/prateek/shared_space/01.mp4"
cap = cv2.VideoCapture(channel)
batch =[]
op_label = ""
i = 0
while True:
    ret,frame = cap.read()
    if not ret:
        break
    frame_res = cv2.resize(frame,(img_width,img_height))
    batch.append(frame_res)
    if len(batch)==seq_len:
        op_label =  classes[np.argmax(model.predict(np.expand_dims(batch,axis=0)))]
        i+=1
        batch = []
    cv2.putText(frame,"updated_label "+str(i)+" = "+op_label,(10,40),cv2.FONT_HERSHEY_COMPLEX,.5,(0,0,0),1,cv2.LINE_4)
    cv2.imshow("preview",frame)
    k = cv2.waitKey(1)
    if k==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()
        
